# This notebook serves as an example of how to create AutoTST objects and how to create 3D geometries

In [1]:
#General imports
import os, sys
import logging
from copy import deepcopy
import numpy as np
import pandas as pd
from multiprocessing import Process

#RDKit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.Pharm3D import EmbedLib
from rdkit import DistanceGeometry
from rdkit.Chem import rdDistGeom

#ASE imports
import ase
from ase import Atom, Atoms
from ase.calculators.gaussian import Gaussian as ASEGaussian
from ase.visualize import view

#RMG-Py imports
from rmgpy.molecule import Molecule as RMGMolecule
from rmgpy.species import Species as RMGSpecies
from rmgpy.reaction import Reaction as RMGReaction

#AutoTST imports
from autotst.species import Species, Conformer
from autotst.reaction import Reaction, TS 
from autotst.calculator.gaussian import Gaussian
from autotst.calculator.statmech import StatMech
from autotst.job.job import Job


Using Theano backend.


# Creating `Species` and `Conformer` objects

Each stable reactant and product species are generated as `Species` objects. These objects can contain many `Conformer` objects that can represent multiple resonance structures. 
`Species` and `Conformer` objects can be initialized as follows:
- `Species(["SMILES_STRING"])`
- `Conformer("SMILES_STRING")`

For a `Species` object, it can take lists of SMILES strings because there may be multiple resonance structures for a given SMILES structure. However, `Conformer` objects represent only a single isomer.
`Species` objects can contain multiple `Conformer` and can be 

In [2]:
smiles = ["[CH2]C=C(C)C"]
for smile in smiles:
    molecule = RMGMolecule(SMILES=smile)


In [3]:
species = Species(["[CH2]C=C(C)C"])
print("For {} there are {} resonance structures".format(species, len(species.conformers)))
print("These structures are:")
for smiles in species.conformers.keys():
    print ("\t- {}".format(smiles))

For <Species "[CH2]C=C(C)C / C=C[C](C)C"> there are 2 resonance structures
These structures are:
	- C=C[C](C)C
	- [CH2]C=C(C)C


In [4]:
species.conformers

{'C=C[C](C)C': [<Conformer "C=C[C](C)C">],
 '[CH2]C=C(C)C': [<Conformer "[CH2]C=C(C)C">]}

When looking at a specific `Conformer` object, you need to know that a `Species` contants a dictionary in `Species.conformers`. The keys are the possible SMILES strings associated with the species and the values are lists of `Conformer` objects for that specific SMILES. When initialized, these lists will be length 1 but will be extended if you generate more conformers. Below is how to view a single `Conformer` object 

In [5]:
conformer = species.conformers[smiles][0]
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Creating `Reaction` and `TS` objects
For this example we will be looking at a hydrogen abstraction reaction by peroxyl radical of 2-methylbut-2-ene

First, you need to initialize the autotst `Reaction` object as done below. This can be done by using either reaction strings that look like `r1+r2_p1+p2` where `r1`, `r2`, `p1`, and `p2` are smiles strings for the molecules involved in the reaction. In addition, not all reaction need to by bimolecular. AutoTST currently supports reactions of the following reaction families:
- Hydrogen Abstraction (`H_Abstraction`: `r1+r2_p1+p2`)
- Intra hydrogen migration (`intra_H_migration`: `r1_p1`)
- R addition to multiple bond (`R_Addition_MultipleBond`: `r1+r2_p1`)

You can specify a `reaction_family` in a `Reaction` object, however, it is not needed as AutoTST will attempt to match the reaction provided to one of the three supported reaction families.
Alternatively, you can intialize a `Reaction` object using an `RMGReaction` object. This can be done as follows:

`rxn = Reaction(rmg_reaction=RMGReaction())`

The initialization of the reaction will also create a forward and a reverse transition state geometry. And these can be visualized using `py3dmol`.

`Reaction` objects are similar to `Species` objects in that they both contain a dictionary corresponding to their conformers. The `Reaction.ts` is the dictionary of transition states for a reaction just like the `Species.conformers` is the dictionary of conformers for a species. The only difference is the keys for the `Reaction.ts` are simply `"forward"` and `"reverse"` to denote the `TS` generated in either direction

In [6]:
rxn = Reaction(label="CC=C(C)C+[O]O_[CH2]C=C(C)C+OO")

reaction.py:231 load_databases INFO Loading RMG database from '/Users/nathan/Code/RMG-database/input'
thermo.py:840 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:840 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:840 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:857 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
statmech.py:529 loadGroups INFO Loading frequencies group database from /Users/nathan/Code/RMG-database/input/statmech/groups...
base.py:220 load INFO Loading transitions state family 

In [7]:
transitionstates = rxn.ts["reverse"] #the rxn.ts is a dictionary with keys being "forward" and "reverse"
ts = transitionstates[0] #transitionstates is a list of TS objects, this list is currently length 1
rxn.ts

reaction.py:309 generate_distance_data INFO The distance data is as follows: 
DistanceData(distances={'d12': 1.322228,'d13': 2.547443,'d23': 1.235329,}, uncertainties={'d12': 0.368943,'d13': 0.209844,'d23': 0.312795,}, comment='Matched node C/H3/Cd ([<Entry index=119 label="C/H3/Cd">, <Entry index=210 label="Csj/Ct/H2">])\nMatched node OjO ([<Entry index=126 label="C/H2/Cd/Cd">, <Entry index=32 label="OjO">])\n')
reaction.py:719 get_rdkit_mol INFO Initially embedded molecule
reaction.py:719 get_rdkit_mol INFO Initially embedded molecule
reaction.py:724 get_rdkit_mol INFO Getting bounds matrix
reaction.py:759 get_bounds_matrix INFO before
reaction.py:762 get_bounds_matrix INFO Got bounds matrix
reaction.py:729 get_rdkit_mol INFO Editing bounds matrix
reaction.py:779 set_limits INFO For atoms 4 and 7 we have a distance of: 	 1.322228
reaction.py:779 set_limits INFO For atoms 7 and 1 we have a distance of: 	 1.235329
reaction.py:779 set_limits INFO For atoms 4 and 1 we have a distance of:

{'forward': [<TS "CC=C(C)C.[O]O">], 'reverse': [<TS "OO.[CH2]C=C(C)C">]}

# Editing geometries of `Conformer` and `TS` objects

AutoTST allows you to edit the following features of a `Conformer` or a `TS` object:
- Bond length
- Angles
- Dihedrals
- CisTrans bond orientation
- Sterocenter orientation

The definitions for all of these objects can be found in `geometry.py`.

In [8]:
conformer = Conformer("ClC=C(O)C(N)Cl")
conformer.view()
print("{} has the following geometries:".format(conformer))
print("")
print("Bonds")
for bond in conformer.bonds:
    print("\t- {}: {}".format(bond, bond.index))
    
print("")
print("Angles")
for angle in conformer.angles:
    print("\t- {}: {}".format(angle, angle.index))
    
print("")
print("Dihedrals")
for torsion in conformer.torsions:
    print("\t- {}: {}".format(torsion, torsion.index))
    
print("")
print("CisTrans Bonds")
for cistran in conformer.cistrans:
    print("\t- {}: ".format(cistran, cistran.index))
    
print("")
print("Chiral Centers")
for chiral_center in conformer.chiral_centers:
    print("\t- {}: {}".format(chiral_center, chiral_center.index))


You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Conformer "ClC=C(O)C(N)Cl"> has the following geometries:

Bonds
	- <Bond "(0, 4)">: 0
	- <Bond "(1, 6)">: 1
	- <Bond "(2, 5)">: 2
	- <Bond "(2, 11)">: 3
	- <Bond "(3, 8)">: 4
	- <Bond "(3, 9)">: 5
	- <Bond "(3, 4)">: 6
	- <Bond "(4, 5)">: 7
	- <Bond "(4, 7)">: 8
	- <Bond "(5, 6)">: 9
	- <Bond "(6, 10)">: 10

Angles
	- <Angle "(0, 4, 3)">: 0
	- <Angle "(0, 4, 5)">: 1
	- <Angle "(0, 4, 7)">: 2
	- <Angle "(1, 6, 5)">: 3
	- <Angle "(1, 6, 10)">: 4
	- <Angle "(2, 5, 4)">: 5
	- <Angle "(2, 5, 6)">: 6
	- <Angle "(3, 4, 5)">: 7
	- <Angle "(3, 4, 7)">: 8
	- <Angle "(4, 3, 8)">: 9
	- <Angle "(4, 3, 9)">: 10
	- <Angle "(4, 5, 6)">: 11
	- <Angle "(5, 2, 11)">: 12
	- <Angle "(5, 4, 7)">: 13
	- <Angle "(5, 6, 10)">: 14
	- <Angle "(8, 3, 9)">: 15

Dihedrals
	- <Torsion "(11, 2, 5, 6)">: 0
	- <Torsion "(9, 3, 4, 7)">: 1
	- <Torsion "(7, 4, 5, 6)">: 2

CisTrans Bonds
	- <CisTrans "[4, 5, 6, 10] - Z">: 

Chiral Centers
	- <ChiralCenter "4 - ?">: 0


In addition, you can set variables for each of these objects. This next section of the notebook will walk though how to do all of this.

In [9]:
conformer.set_bond_length(bond_index=3, length=3) #Length is specified in angstroms
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
conformer.set_angle(angle_index=7, angle=45.) #Angles are specified in degrees
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
conformer.set_torsion(torsion_index=0, dihedral=180)
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
conformer.set_cistrans(cistrans_index=0, stero="z") #options for stero are E and Z
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
conformer.set_chirality(chiral_center_index=0, stero="r")
conformer.set_chirality??
conformer.view()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## AND all of these features can be extended to `TS` objects as well 

# Now, let's look at how to generate ensembles of conformers for `Species` and `Reactions` using AutoTST

Both `Species` and `Reaction` objects have a built-in method called `generate_conformers` and this method takes one argument of an ASE calculator object. This method will then generate all possible combinations of dihedrals, CisTrans bonds, and chiral centers to identify every possible conformer. All of these conformers are then optimized using ASE's BFGS optimizer and the calculator that a user provides. From this, a list of conformers within 1 kcal/mol are generated and added to the `Species.conformers` or the `Reaction.ts` dict. Below are a few examples.

In [14]:
from hotbit import Hotbit
species = Species(["CCCC"])
species.generate_conformers(calculator=Hotbit())
species.conformers







 _           _    _     _ _
| |__   ___ | |_ | |__ |_| |_
|  _ \ / _ \|  _||  _ \| |  _|
| | | | ( ) | |_ | ( ) | | |_
|_| |_|\___/ \__|\____/|_|\__|  ver. 0.1 (svn=Unversioned directory)
Distributed under GNU GPL; see /Users/nathan/Code/hb/lib/python/LICENSE
Date: Thu May  2 16:57:19 2019
Nodename: Nathans-MacBook.local
Arch: x86_64
Dir: /Users/nathan/Code/AutoTST/ipython_notebooks
System: H10C4
       Charge= 0.0
       Container Bravais: pbc:[0,0,0], cell:[0.00,0.00,0.00] Ang, 
   cosines(12,13,23):[nan,nan,nan]
Symmetry operations (if any):
       0: pbc= False , kappa-points=1, M=1
       1: pbc= False , kappa-points=1, M=1
       2: pbc= False , kappa-points=1, M=1
Electronic temperature: 0.02 eV
Mixer: Anderson with memory = 3 , mixing parameter = 0.2
14 atoms, 26 states, 26.0 electrons (13.0 filled states)
Initial n ranges:[0,0] [0,0] [0,0] 
Element C in /Users/nathan/Code/hb/param/C.elm
    *no comment
Element H in /Users/nathan/Code/hb/param/H.elm
    *Standard H-atom



systematic.py:192 systematic_search INFO There are 6 unique conformers generated


Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Hamiltonian ~92.899 % filled.
Hamiltonian ~92.899 % filled.
Hamiltonian ~92.899 % filled.
Hamiltonian ~92.899 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~92.899 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~92.899 % filled.


systematic.py:287 systematic_search INFO We have identified 5 unique conformers for <Conformer "CCCC">


Solved zero times.

Timing:
            label                    time     calls    %sub  %covered   %tot
-------------------------------------------------------------------------------
Hotbit                              21.267         1 (100.0 %,  1.4 %) 100.0 % |000000000000000000000000000000|
../initialization                    0.303         1 (  1.4 %        )   1.4 % ||
 -------------------------------------------------------------------------------
total time       21.267 seconds      0 h  0 min  21 sec
Thu May  2 16:57:40 2019


{'CCCC': [<Conformer "CCCC">,
  <Conformer "CCCC">,
  <Conformer "CCCC">,
  <Conformer "CCCC">,
  <Conformer "CCCC">]}

In [15]:
reaction = Reaction("C+[O]O_[CH3]+OO")
reaction.generate_conformers(calculator=Hotbit())
reaction.ts

reaction.py:394 get_labeled_reaction INFO Matched reaction to H_Abstraction family
species.py:118 __init__ INFO Could not generate resonance structures for this species... Using molecule provided
species.py:118 __init__ INFO Could not generate resonance structures for this species... Using molecule provided
reaction.py:304 generate_distance_data INFO Distance between *1 and *3 is too small, setting it to lower bound of uncertainty
reaction.py:309 generate_distance_data INFO The distance data is as follows: 
DistanceData(distances={'d12': 1.415393,'d13': 2.434963,'d23': 1.127452,}, uncertainties={'d12': 0.368199,'d13': 0.209098,'d23': 0.288777,}, comment='Matched node C_methane ([<Entry index=43 label="C_methane">, <Entry index=103 label="CsjCH2">])\nMatched node OjO ([<Entry index=126 label="C/H2/Cd/Cd">, <Entry index=32 label="OjO">])\n')
reaction.py:719 get_rdkit_mol INFO Initially embedded molecule
reaction.py:719 get_rdkit_mol INFO Initially embedded molecule
reaction.py:724 get_rd







 _           _    _     _ _
| |__   ___ | |_ | |__ |_| |_
|  _ \ / _ \|  _||  _ \| |  _|
| | | | ( ) | |_ | ( ) | | |_
|_| |_|\___/ \__|\____/|_|\__|  ver. 0.1 (svn=Unversioned directory)
Distributed under GNU GPL; see /Users/nathan/Code/hb/lib/python/LICENSE
Date: Thu May  2 16:57:41 2019
Nodename: Nathans-MacBook.local
Arch: x86_64
Dir: /Users/nathan/Code/AutoTST/ipython_notebooks
System: H5CO2
       Charge= 0.0
       Container Bravais: pbc:[0,0,0], cell:[0.00,0.00,0.00] Ang, 
   cosines(12,13,23):[nan,nan,nan]
Symmetry operations (if any):
       0: pbc= False , kappa-points=1, M=1
       1: pbc= False , kappa-points=1, M=1
       2: pbc= False , kappa-points=1, M=1
Electronic temperature: 0.02 eV
Mixer: Anderson with memory = 3 , mixing parameter = 0.2
8 atoms, 17 states, 21.0 electrons (10.5 filled states)
Initial n ranges:[0,0] [0,0] [0,0] 
Element O in /Users/nathan/Code/hb/param/O.elm
    *no comment
Element C in /Users/nathan/Code/hb/param/C.elm
    *no comment
Element

systematic.py:287 systematic_search INFO We have identified 4 unique conformers for <TS "C.[O]O">
systematic.py:192 systematic_search INFO There are 17 unique conformers generated


Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Hamiltonian ~87.543 % filled.
Hamiltonian ~87.543 % filled.
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % filled.
Memory consumption estimate: > 0.00 GB
Memory consumption estimate: > 0.00 GB
Hamiltonian ~87.543 % fil

systematic.py:287 systematic_search INFO We have identified 6 unique conformers for <TS "OO.[CH3]">


Solved zero times.

Timing:
            label                    time     calls    %sub  %covered   %tot
-------------------------------------------------------------------------------
Hotbit                             100.406         1 (100.0 %,  0.3 %) 100.0 % |000000000000000000000000000000|
../initialization                    0.327         1 (  0.3 %        )   0.3 % ||
 -------------------------------------------------------------------------------
total time      100.406 seconds      0 h  1 min  40 sec
Thu May  2 16:59:21 2019


{'forward': [<TS "C.[O]O">, <TS "C.[O]O">, <TS "C.[O]O">, <TS "C.[O]O">],
 'reverse': [<TS "OO.[CH3]">,
  <TS "OO.[CH3]">,
  <TS "OO.[CH3]">,
  <TS "OO.[CH3]">,
  <TS "OO.[CH3]">,
  <TS "OO.[CH3]">]}

For right now, we're going to be looking at the first conformers for the `Species` and `Reaction` listed above.

In [16]:
ts = reaction.ts["forward"][0]
conformer = species.conformers["CCCC"][0]

# Now, to look at writing input files for `Conformer` and `TS` objects

This is realitively easy, you just need to use the `Gaussian` calculator object to do all of this. This starts with an initialzation of the object followed by calling it's methods on different objects to get ASE calculator objects that can write your geometries.

The main methods of the `Gaussian` calculator are:
- For species conformers 
    - `get_conformer_calc`
- For transition states
    - `get_shell_calc`
    - `get_center_calc`
    - `get_overall_calc`
    - `get_irc_calc`
- For both
    - `get_rotor_calc`

In [17]:
gaussian = Gaussian()

For `Conformer` objects

In [18]:

calc = gaussian.get_conformer_calc(conformer=conformer)
calc.write_input(conformer.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

CCCC_0

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ opt=(calcfc,verytight,gdiis,maxcycles=900) freq IOP(7/33=1,2/16=3) scf=(maxcycle=900) 

Gaussian input prepared by ASE

0 1
C                 0.7354773892       -0.5134376003       -0.4435545991
C                -0.6152275702       -0.7833727200        0.1347266939
C                 1.4708785306        0.6001543075        0.2570064789
C                -1.5576774464        0.3717083329       -0.0133286741
H                 1.3514541492       -1.4408514250       -0.3998148131
H                 0.6384864521       -0.2543841110       -1.5085983108
H                -1.0592735869       -1.6822519107       -0.3636475485
H                -0.5140400339       -1.0467673405        1.2016221745
H                 0.9226907733        1.5490177860        0.1594326502
H                 1.5916942864        0.3858269094        1.3352985542
H                 2.4876999397        0.7411498210       -0.1864867196
H                -1.9843555104        0.40100

For `TS` objects

In [19]:
calc = gaussian.get_shell_calc(ts=ts)
calc.write_input(ts.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

C+[O]O_[CH3]+OO_forward_shell_0

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ Opt=(ModRedun,Loose,maxcycles=900) Int(Grid=SG1) scf=(maxcycle=900) 

Gaussian input prepared by ASE

0 2
O                 3.1075007166        0.0366083817        1.1380898417
O                 2.7719170395       -0.9838382949        0.3778717969
C                 1.8945968834        0.2679645289       -1.5076958826
H                 2.0535769134       -0.5567744322       -0.3833424702
H                 2.1299086782       -0.3061419699       -2.4273764453
H                 0.8496485340        0.6404518280       -1.5151018938
H                 2.6179912682        1.0940032327       -1.3491742317
H                 3.9834411235        0.3211441193        0.7742050968

3 4 F
4 2 F
3 4 2 F



In [20]:
calc = gaussian.get_center_calc(ts=ts)
calc.write_input(ts.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

C+[O]O_[CH3]+OO_forward_center_0

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ Opt=(ts,calcfc,noeigentest,ModRedun,maxcycles=900) scf=(maxcycle=900) 

Gaussian input prepared by ASE

0 2
O                 3.1075007166        0.0366083817        1.1380898417
O                 2.7719170395       -0.9838382949        0.3778717969
C                 1.8945968834        0.2679645289       -1.5076958826
H                 2.0535769134       -0.5567744322       -0.3833424702
H                 2.1299086782       -0.3061419699       -2.4273764453
H                 0.8496485340        0.6404518280       -1.5151018938
H                 2.6179912682        1.0940032327       -1.3491742317
H                 3.9834411235        0.3211441193        0.7742050968

1 5 F
1 6 F
1 7 F
1 8 F
5 6 F
5 7 F
5 8 F
6 7 F
6 8 F
7 8 F



In [21]:
calc = gaussian.get_overall_calc(ts=ts)
calc.write_input(ts.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

C+[O]O_[CH3]+OO_forward_0

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ opt=(ts,calcfc,noeigentest,maxcycles=900) freq scf=(maxcycle=900) IOP(7/33=1,2/16=3) 

Gaussian input prepared by ASE

0 2
O                 3.1075007166        0.0366083817        1.1380898417
O                 2.7719170395       -0.9838382949        0.3778717969
C                 1.8945968834        0.2679645289       -1.5076958826
H                 2.0535769134       -0.5567744322       -0.3833424702
H                 2.1299086782       -0.3061419699       -2.4273764453
H                 0.8496485340        0.6404518280       -1.5151018938
H                 2.6179912682        1.0940032327       -1.3491742317
H                 3.9834411235        0.3211441193        0.7742050968





In [22]:
calc = gaussian.get_irc_calc(ts=ts)
calc.write_input(ts.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

C+[O]O_[CH3]+OO_irc_forward_0

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ irc=(calcall) 

Gaussian input prepared by ASE

0 2
O                 3.1075007166        0.0366083817        1.1380898417
O                 2.7719170395       -0.9838382949        0.3778717969
C                 1.8945968834        0.2679645289       -1.5076958826
H                 2.0535769134       -0.5567744322       -0.3833424702
H                 2.1299086782       -0.3061419699       -2.4273764453
H                 0.8496485340        0.6404518280       -1.5151018938
H                 2.6179912682        1.0940032327       -1.3491742317
H                 3.9834411235        0.3211441193        0.7742050968





For hindered rotor calculations

In [23]:
torsion = conformer.torsions[0]
calc = gaussian.get_rotor_calc(conformer=conformer, torsion=torsion)
calc.write_input(conformer.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

CCCC_36by10_0_2

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ Opt=(CalcFC,ModRedun) 

Gaussian input prepared by ASE

0 1
C                 0.7354773892       -0.5134376003       -0.4435545991
C                -0.6152275702       -0.7833727200        0.1347266939
C                 1.4708785306        0.6001543075        0.2570064789
C                -1.5576774464        0.3717083329       -0.0133286741
H                 1.3514541492       -1.4408514250       -0.3998148131
H                 0.6384864521       -0.2543841110       -1.5085983108
H                -1.0592735869       -1.6822519107       -0.3636475485
H                -0.5140400339       -1.0467673405        1.2016221745
H                 0.9226907733        1.5490177860        0.1594326502
H                 1.5916942864        0.3858269094        1.3352985542
H                 2.4876999397        0.7411498210       -0.1864867196
H                -1.9843555104        0.4010061669       -1.0420713857
H                -1.040017645

In [24]:
torsion = ts.torsions[0]
calc = gaussian.get_rotor_calc(conformer=ts, torsion=torsion)
calc.write_input(ts.ase_molecule)
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

C+[O]O_[CH3]+OO_36by10_0_1

%nprocshared=20
%mem=5GB
#p m062x/cc-pVTZ Opt=(ts,CalcFC,ModRedun) 

Gaussian input prepared by ASE

0 2
O                 3.1075007166        0.0366083817        1.1380898417
O                 2.7719170395       -0.9838382949        0.3778717969
C                 1.8945968834        0.2679645289       -1.5076958826
H                 2.0535769134       -0.5567744322       -0.3833424702
H                 2.1299086782       -0.3061419699       -2.4273764453
H                 0.8496485340        0.6404518280       -1.5151018938
H                 2.6179912682        1.0940032327       -1.3491742317
H                 3.9834411235        0.3211441193        0.7742050968

B 1 8
B 1 2
B 3 5
B 3 6
B 3 4
B 3 7
B 4 2
D 8 1 2 4 S 36 10.0
D 7 3 4 2 F
D 3 4 2 1 F



At this point, you now have a way to have input files for quantum chemistry optimizations to be automatically written for you. And you can have all of these run automatically using the AutoTST `Job` class